# clover-on-clover connected proton charge radius

In [1]:
import numpy as np
import pandas as pd
import gvar as gv
import lsqfit
import matplotlib.pyplot as plt
%matplotlib notebook
from tqdm import tqdm_notebook as tqdm
import pcr_lib as pcr

# switches

In [2]:
switches = dict()
# PLOTTING OPTIONS
switches['plot'] = dict()
switches['plot']['correlator'] = False
# FITTING OPTIONS
switches['fit'] = dict()
switches['fit']['switch'] = True # turn fitting on/off
# choose dataset
switches['fit']['clist']   = ['nucleon'] #,'dnucleon','gV','dgV'] # the derivatives should be fit with the correlators or else there's no handel on the parameters
switches['fit']['snk_src'] = ['SS']
switches['fit']['mom']     = [0,1,2] # n in 2*n*pi/L
switches['fit']['nstate']  = 2
# time region
switches['fit']['T_3pt']   = [10,14] # t_snk for 3pt: only 10 or 14
switches['fit']['trange'] = dict()
switches['fit']['trange']['nucleon_tmin']   = [5,5] # snk location
switches['fit']['trange']['nucleon_tmax']   = [10,10] # snk location
switches['fit']['trange']['dnucleon_tmin']   = [2,2] # snk location
switches['fit']['trange']['dnucleon_tmax']   = [10,10] # snk location
switches['fit']['trange']['gV_tmin']   = [2,2] # current insertion time
switches['fit']['trange']['gV_tmax']   = [9,9] # current insertion time
switches['fit']['trange']['dgV_tmin']   = [2,2] # current insertion time
switches['fit']['trange']['dgV_tmax']   = [9,9] # current insertion time
# PRIORS
switches['p'] = dict()
# nucleon q = 0
switches['p']['E0_q0'] = gv.gvar(0.7,0.14)
switches['p']['Z0s_q0'] = gv.gvar(4E-8,2E-8)
switches['p']['Z0p_q0'] = gv.gvar(0.0,1.0)
switches['p']['E1_q0'] = gv.gvar(-0.7,0.7) # log splitting to two pions
switches['p']['Z1s_q0'] = gv.gvar(0.0,4E-8)
switches['p']['Z1p_q0'] = gv.gvar(0.0,1.0)
# nucleon q = 1
switches['p']['E0_q1'] = gv.gvar(0.75,0.15)
switches['p']['Z0s_q1'] = gv.gvar(6E-8,3E-8)
switches['p']['Z0p_q1'] = gv.gvar(0.0,1.0)
switches['p']['E1_q1'] = gv.gvar(-0.7,0.7)
switches['p']['Z1s_q1'] = gv.gvar(0.0,6E-8)
switches['p']['Z1p_q1'] = gv.gvar(0.0,1.0)
# nucleon q = 2
switches['p']['E0_q2'] = gv.gvar(0.85,0.17)
switches['p']['Z0s_q2'] = gv.gvar(7E-8,3.5E-8)
switches['p']['Z0p_q2'] = gv.gvar(0.0,1.0)
switches['p']['E1_q2'] = gv.gvar(-0.7,0.7)
switches['p']['Z1s_q2'] = gv.gvar(0.0,3.5E-8)
switches['p']['Z1p_q2'] = gv.gvar(0.0,1.0)
# dnucleon q = 0 (Y comes before Z... so it's a derivative? something like that)
switches['p']['Y0s_q0'] = gv.gvar(3.2E-7,1.6E-7)
switches['p']['Y0p_q0'] = gv.gvar(0.0,1.0)
switches['p']['Y1s_q0'] = gv.gvar(0.0,3.2E-7)
switches['p']['Y1p_q0'] = gv.gvar(0.0,1.0)
# dnucleon q = 1
switches['p']['Y0s_q1'] = gv.gvar(2.8E-7,1.4E-7)
switches['p']['Y0p_q1'] = gv.gvar(0.0,1.0)
switches['p']['Y1s_q1'] = gv.gvar(0.0,2.8E-7)
switches['p']['Y1p_q1'] = gv.gvar(0.0,1.0)
# dnucleon q = 2
switches['p']['Y0s_q2'] = gv.gvar(1.8E-7,0.9E-7)
switches['p']['Y0p_q2'] = gv.gvar(0.0,1.0)
switches['p']['Y1s_q2'] = gv.gvar(0.0,1.8E-7)
switches['p']['Y1p_q2'] = gv.gvar(0.0,1.0)
# gV q = 0
switches['p']['G00_q0'] = gv.gvar(0.0,1.0)
switches['p']['G10_q0'] = gv.gvar(0.0,1.0)
switches['p']['G11_q0'] = gv.gvar(0.0,1.0)
# gV q = 1
switches['p']['G00_q1'] = gv.gvar(0.0,1.0)
switches['p']['G10_q1'] = gv.gvar(0.0,1.0)
switches['p']['G11_q1'] = gv.gvar(0.0,1.0)
# gV q = 2
switches['p']['G00_q2'] = gv.gvar(0.0,1.0)
switches['p']['G10_q2'] = gv.gvar(0.0,1.0)
switches['p']['G11_q2'] = gv.gvar(0.0,1.0)
# dgV q = 0
switches['p']['F00_q0'] = gv.gvar(0.0,1.0)
switches['p']['F10_q0'] = gv.gvar(0.0,1.0)
switches['p']['F11_q0'] = gv.gvar(0.0,1.0)
# dgV q = 1
switches['p']['F00_q1'] = gv.gvar(0.0,1.0)
switches['p']['F10_q1'] = gv.gvar(0.0,1.0)
switches['p']['F11_q1'] = gv.gvar(0.0,1.0)
# dgV q = 2
switches['p']['F00_q2'] = gv.gvar(0.0,1.0)
switches['p']['F10_q2'] = gv.gvar(0.0,1.0)
switches['p']['F11_q2'] = gv.gvar(0.0,1.0)

# read csv file into dataframe
## ensemble information
+ 0.12fm lattice spacing
+ pion mass 400 MeV
+ isoclover

## configuration information
+ 1000 to 2950 skip 10 (196 configurations)
    + configuration 1470 is missing for baryon 2pt tsrc 48, idx = 47
+ two point correlation function
    + `bar_s22_SS_tsrc_[0,48].dat.[re,im]`
    + two sources at `[0,48]`
    + each line contains:
        + individual momentum `[0,1,-1,2,-2]`
        + individual time slice
        + array of z-dependence
        + *e.g.* `bar_s22_S0src_S0_px0_py0_pz0_t0_re`
+ three point correlation function
    + `barff_s22_SS_g8_T[10,14]_tsrc_[0,48].dat.[re,im]`
    + `g8` corresponds to the `V4` current insertion
    + `T[10,14]` is snk location
    + two sources at `[0,48]`
    + each line contains:
        + individual momentum transfer (snk at zero momentum) `[0,1,-1,2,-2]`
        + individual time slice
        + array of z-dependence at current insertion
        + *e.g.* `barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_t0_re`

## strategy
+ read zero momentum spatial correlator
+ perform source average
+ construct two and three point (with momentum transfer)
+ construct moment correlators

## notes
+ read SS src snk

In [3]:
zdata = pcr.read_csv(switches)

2pt: bar_s22_S0src_S0_px0_py0_pz0_re (195, 96, 32)


# construct two and three point correlators
+ Lattice 2016 proceedings @ https://arxiv.org/pdf/1610.02354.pdf
+ project to definite momentum

In [4]:
data = dict()
# two point correlation function
if 'nucleon' in switches['fit']['clist']:
    for snk_src in switches['fit']['snk_src']:
        snk = snk_src[0]
        src = snk_src[1]
        for n in switches['fit']['mom']:
            k = 2.*np.pi*n/32
            pos = zdata['bar_s22_%s0src_%s0_px0_py0_pz0_re' %(src,snk)]*np.cos(k*np.arange(32))
            neg = zdata['bar_s22_%s0src_%s0_px0_py0_pz0_re' %(src,snk)]*np.cos(-k*np.arange(32))
            data['nucleon_%s_q%s' %(str(snk_src),str(n))] = np.sum((pos+neg)/2,axis=2) # mom avg & project to 3 momentum
# gV correlation function
if 'gV' in switches['fit']['clist']:
    for snk_src in switches['fit']['snk_src']:
        snk = snk_src[0]
        src = snk_src[1]
        for n in switches['fit']['mom']:
            k = 2.*np.pi*n/32
            for T in switches['fit']['T_3pt']:
                pos = zdata['barff_s2_%s0src_%s0snk_T%s_px0_py0_pz0_g8_re' %(src,snk,str(T))]*np.cos(k*np.arange(32))
                neg = zdata['barff_s2_%s0src_%s0snk_T%s_px0_py0_pz0_g8_re' %(src,snk,str(T))]*np.cos(-k*np.arange(32))
                data['gV_%s_T%s_q%s' %(str(snk_src),str(T),str(n))] = np.sum((pos+neg)/2,axis=2) # mom avg & project to 3 momentum
data.keys()

dict_keys(['nucleon_SS_q0', 'nucleon_SS_q1', 'nucleon_SS_q2'])

# construct moments of correlation functions

In [5]:
# two point moment data
z = np.roll(np.arange(32)-15,17)
rz2 = z**2
# apply moment
# output is Ncf * T * Z
if 'dnucleon' in switches['fit']['clist']:
    for q in switches['fit']['mom']:
        for snk_src in switches['fit']['snk_src']:
            snk = snk_src[0]
            src = snk_src[1]
            if q == 0:
                D = 0.5*rz2
            else:
                k = 2.*np.pi*q/32
                D = z/(2.*k)*np.sin(k*np.arange(32))
            if 'dnucleon' in switches['fit']['clist']:
                temp = D*zdata['bar_s22_%s0src_%s0_px0_py0_pz0_re' %(src,snk)]
                # UNCOMMENT TO PLOT Z DEPENDENCE
                #pcr.plot_2pt_z(temp,title='2pt') #title='dnucleon_%s_q%s' %(snk_src,str(q)))
                # project to 3 momentum
                data['dnucleon_%s_q%s' %(snk_src,str(q))] = np.sum(temp,axis=2)
            if 'dgV' in switches['fit']['clist']:
                for T in switches['fit']['T_3pt']:
                    temp = D*zdata['barff_s2_%s0src_%s0snk_T%s_px0_py0_pz0_g8_re' %(src,snk,str(T))]
                    # UNCOMMENT TO PLOT Z DEPENDENCE
                    #pcr.plot_3pt_z(temp,title='3pt') #title='dgV_%s_T%s_q%s' %(snk_src,str(T),str(q)))
                    # project 3 momentum
                    data['dgV_%s_T%s_q%s' %(snk_src,str(T),str(q))] = np.sum(temp,axis=2)
print(data.keys())

dict_keys(['nucleon_SS_q0', 'nucleon_SS_q1', 'nucleon_SS_q2'])


# construct correlated dataset

In [6]:
gvdata = gv.dataset.avg_data(data)
for k in gvdata.keys():
    print(k,np.shape(gvdata[k]))

nucleon_SS_q0 (96,)
nucleon_SS_q1 (96,)
nucleon_SS_q2 (96,)


# plot correlation function

In [7]:
dat2name = [i for i in gvdata.keys() if i.split('_')[0] in ['nucleon']]
datd2name = [i for i in gvdata.keys() if i.split('_')[0] in ['dnucleon']]
if switches['plot']['correlator']:
    # plot effective mass correlator
    fig = plt.figure('meff',figsize=(7,4.326237))
    ax = plt.axes([0.15,0.15,0.8,0.8])
    x = np.arange(2,18)
    for d in dat2name:
        meff = np.log(gvdata[d]/np.roll(gvdata[d],-1))[x]
        ax.errorbar(x,y=[i.mean for i in meff],yerr=[i.sdev for i in meff],label=d)
    ax.legend()
    plt.draw()
    # plot scaled two point correlator (SS ONLY NOW)
    fit = plt.figure('scaled correlator',figsize=(7,4.326237))
    ax = plt.axes([0.15,0.15,0.8,0.8])
    x = np.arange(2,11)
    for d in dat2name:
        meff = np.log(gvdata[d]/np.roll(gvdata[d],-1))
        scor = (gvdata[d]*2*meff*np.exp(meff*np.arange(len(meff))))[x]
        ax.errorbar(x,y=[i.mean for i in scor],yerr=[i.sdev for i in scor],label=d)
    ax.legend()
    plt.draw()
    # plot two point slope (SS ONLY NOW)
    fit = plt.figure('two point slope',figsize=(7,4.326237))
    ax = plt.axes([0.15,0.15,0.8,0.8])
    x = np.arange(2,11)
    for dd in datd2name:
        d = dd[1:] # two point name (take out derivative)
        meff = np.log(gvdata[d]/np.roll(gvdata[d],-1))
        scor = gvdata[d]*2*meff*np.exp(meff*np.arange(len(meff)))
        dcor = ((gvdata[dd]/gvdata[d]+0.5/meff**2+0.5*np.arange(len(meff))/meff)*0.5*scor)[x]
        ax.errorbar(x,y=[i.mean for i in dcor],yerr=[i.sdev for i in dcor],label=d)
    ax.legend()
    plt.draw()
    # plot three point correlator (SS ONLY NOW)
    dat3name = ['gV_SS_T10_q0','dgV_SS_T10_q0']
    fig = plt.figure('correlator',figsize=(7,4.326237))
    ax = plt.axes([0.15,0.15,0.8,0.8])
    x = np.arange(0,16)
    for d in dat3name:
        corr = gvdata[d][x]
        ax.errorbar(x,y=[i.mean for i in corr],yerr=[i.sdev for i in corr])
    plt.draw()

# fit correlators
+ blahblah... I'm too lazy for this... I will regret it later

In [8]:
# instantiate fitter
fitter = pcr.fitter_class(switches['fit']['clist'],switches['fit']['snk_src'],switches['fit']['mom'],switches['fit']['nstate'])
# get priors
p = fitter.p(switches)
# loop over t-range
nucleon_tmin = switches['fit']['trange']['nucleon_tmin']
nucleon_tmax = switches['fit']['trange']['nucleon_tmax']
dnucleon_tmin = switches['fit']['trange']['dnucleon_tmin']
dnucleon_tmax = switches['fit']['trange']['dnucleon_tmax']
gV_tmin = switches['fit']['trange']['gV_tmin']
gV_tmax = switches['fit']['trange']['gV_tmax']
dgV_tmin = switches['fit']['trange']['dgV_tmin']
dgV_tmax = switches['fit']['trange']['dgV_tmax']
for t2min in np.arange(nucleon_tmin[0],nucleon_tmin[1]+1):
    for t2max in np.arange(nucleon_tmax[0],nucleon_tmax[1]+1):
        for dt2min in np.arange(dnucleon_tmin[0],dnucleon_tmin[1]+1):
            for dt2max in np.arange(dnucleon_tmax[0],dnucleon_tmax[1]+1):
                for t3min in np.arange(gV_tmin[0],gV_tmin[1]+1):
                    for t3max in np.arange(gV_tmax[0],gV_tmax[1]+1):
                        for dt3min in np.arange(dgV_tmin[0],dgV_tmin[1]+1):
                            for dt3max in np.arange(dgV_tmax[0],dgV_tmax[1]+1):
                                trange = dict()
                                trange['nucleon'] = np.arange(t2min,t2max+1)
                                trange['dnucleon'] = np.arange(dt2min,dt2max+1)
                                trange['gV'] = np.arange(t3min,t3max+1)
                                trange['dgV'] = np.arange(dt3min,dt3max+1)
                                x = fitter.x(switches,gvdata,trange)
                                y = fitter.y(x,gvdata)
                                if switches['fit']['switch']:
                                    fit = lsqfit.nonlinear_fit(data=(x,y),prior=p,fcn=fitter.fit_function,maxit=1000000)
                                else:
                                    fit = 'turn fitting on'
                                print(fit)

/Users/cchang5/Physics/pcr/pcr_lib.py:150: RuntimeWarning: overflow encountered in exp
  return Zn_snk*Zn_src*np.exp(-En*T) / (2.*En)
/Users/cchang5/Physics/pcr/pcr_lib.py:150: RuntimeWarning: overflow encountered in double_scalars
  return Zn_snk*Zn_src*np.exp(-En*T) / (2.*En)


KeyError: 'undefined key: Z2s_q0'